## Eager Execution

Sources:
- [1] https://www.tensorflow.org/programmers_guide/eager

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
# Check to see eager execution is running
tf.executing_eagerly()

In [ ]:
# Tensorf objects now reference actual values rather than symbolic handles to graph nodes
x = [[2.]]
x = tf.matmul(x, x)

In [ ]:
print(f'The output is {x}')

In [ ]:
a = tf.constant([[1, 2], [3, 4]])
a

In [ ]:
b = tf.add(a, 1)
c = tf.multiply(b, a)
c

In [ ]:
# You can also use numpy with tensors
import numpy as np
c = np.multiply(a, b)
c

In [ ]:
# Get the numpy property from a tensor
a.numpy()

In [ ]:
# Make a custom layer inheriting from the keras layers class
class SimpleLayer(tf.keras.layers.Layer):
    
    def __init__(self, num_outputs):
        self.num_outputs = num_outputs
        
    def build(self, input):
        # Gets called the first time the layer is used
        self.kernel = self.add_variable("kernel", [input.shape[-1], self.num_outputs]) # Input shape dependent
    
    def call(self, input):
        return tf.matmul(input, self.kernel)

In [ ]:
# You can build models with layers by using the Sequential class in keras
simple_model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(784,)),
    tf.keras.layers.Dense(10)
])

In [ ]:
# Build models by inherting from the keras Model class
class SimpleModel(tf.keras.Model):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.layer1 = tf.keras.layers.Dense(10) # Don't need to define input shape due to superclass init
        self.layer2 = tf.keras.layers.Dense(10)
        
    def call(self, input):
        layer1_out = self.layer1(input)
        layer2_out = self.layer2(layer1_out)
        return layer2_out
    
simple_model = SimpleModel

### Eager Training

In [ ]:
tfe = tf.contrib.eager
w = tfe.Variable([[1.0]])

# All forward pass operations get recorded to a tape
with tf.GradientTape() as tape:
    loss = w * w
    
# Play tape backwards to get gradient (tapes can only be used once)
grad = tape.gradient(loss, [w])
grad

In [ ]:
# A Simple example using gradient tape

# A bunch of points randomly spaced around 3*x + 2
NUM_EXAMPLES = 1000
training_input = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_output = 3 * training_input + 2 + noise

# Simple prediction
def prediction(input, weight, bias):
    return input * weight + bias

def loss(weights, biases):
    # Use mse as the loss
    error = prediction(training_input, weights, biases) - training_output
    return tf.reduce_mean(tf.square(error))

# Use gradient tape to return derivative of loss
def grad(weights, biases):
    with tf.GradientTape() as tape:
        loss_value = loss(weights, biases)
    return tape.gradient(loss_value, [weights, biases])

train_steps = 100
learning_rate = 0.01
# Initialize arbitrary weight and bias
W = tfe.Variable(5.)
b = tfe.Variable(10.)

print(f'Initial loss {loss(W, b)}')

# Train
for i in range(train_steps):
    dW, db = grad(W, b)
    W.assign_sub(dW * learning_rate) # Assign and subtract learning rate times gradient
    b.assign_sub(db * learning_rate)
    print(f'Step {i} loss {loss(W, b)}')

print(f'Final values: W = {W.numpy()}, b = {b.numpy()}')

### Train a model

In [ ]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.W = tfe.Variable(5., name='weights')
        self.B = tfe.Variable(10., name='bias')
    def predict(self, input):
        return self.W * input + self.B
    
# Loss and Gradient functions defined with respect to model
def loss(model, inputs, targets):
    error = model.predict(inputs) - targets
    return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, [model.W, model.B])

# Lets create the model and train it
model = Model()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_steps = 100

for i in range(100):
    grads = grad(model, training_input, training_output)
    optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                             global_step=tf.train.get_or_create_global_step())
    print(f'Loss at step {i} is {loss(model, training_input, training_output)}')

### Objects for State

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
# Variables are object and can be un-assigned to remove from memory
with tf.device('cpu:0'):
    v = tfe.Variable(tf.random_normal([10, 10]))
    v = None    

In [ ]:
# Save and restore variables with checkpoints
x = tfe.Variable(10.)

checkpoint = tfe.Checkpoint(x=x)
save_path = checkpoint.save('./ckpt/')
print(save_path)

# modify value of varibale
x.assign(4.)
print(x)

# Restore
checkpoint.restore(save_path)
print(x)